In [1]:
import torch

## PyTorch Workflow 

In [2]:
what_we_are_covering = {
    1: "data (prepare and load",
    2: "build model",
    3: "fitting the model to the  (training)",
    4: "making predictions and evalating a model (inference)", 
    5: "saving and loading a model",
    6: "putting it all together"
                       }

In [3]:
import torch 
from torch import nn # nn contains all of PyTorch's building blocks for neural networks 
import matplotlib.pyplot as plt 

torch.__version__

'2.6.0+cu124'

## 1. Data (preparing and loading)

ML is a game of two parts:
1. get data into a numerical representation
2. build a model to learn patterns in that numerical representation

We'll use a linear regression formula to make a stright line with known **parameters**
1. weight = 0.7
2. bias = 0.3

In [4]:
weight = 0.7 
bias = 0.3 

start = 0
end = 1
step = 0.02 
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias 

# input, output values 
X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [5]:
len(X), len(y)

(50, 50)

## Splitting Data into Training and Test sets
* training set : course material
* validation set : practice exam
* test set : final exam -> test generalization
**Generalization**: the ability for a ML model to perform well on data it hasn't seen befor e

In [6]:
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]

len(X_train), len(y_train), len(X_test), len(y_test)

(40, 40, 10, 10)